In [ ]:
wd <- dirname(dirname(getwd()))
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/helpers/exhaustive_study/exhaustive_help.R"))
library(tidyverse)

#### 0 - Read prepared CPI data

In [ ]:
cpi <- readRDS(paste0(I_DIR, "cpi_go.Rds"))

#### 1 - Scale the data
- scale only numeric features, and avoid scaling response features
- scaling here over all the HMF data, instead of cohort specific

In [ ]:
scale_the_data <- function( df ){
    col_types <- sapply( df, typeof )
    responses <- colnames(df %>% select(Y_best_response_binary, Y_best_response_time_in_days, Y_relapse, contains("Surv")))
    step1 <- col_types[which(col_types != "character")]
    step2 <- names(step1[-which(names(step1) %in% responses)])
    df %>% mutate_at( vars(all_of(step2)), ~ (scale(.) %>% as.vector))
}
cpi_scaled <- scale_the_data(cpi)

#### 2 - Create Data Store for Exhaustive Analysis
- Store list of dataframes to be used in exhaustive analyses

In [ ]:
studies <- list()

studies[['all']] <- cpi_scaled
studies[['skin']]    <- cpi_scaled %>% filter(clinical_tumor_location_group == "skin")
studies[['lung']]    <- cpi_scaled %>% filter(clinical_tumor_location_group == "lung")
studies[['bladder']] <- cpi_scaled %>% filter(clinical_tumor_location_group == "bladder")
studies[['other']]   <- cpi_scaled %>% filter(clinical_tumor_location_group == "other")

- Prepare for best response and survival studies

In [ ]:
get_ready <- function ( studies ) {

    response <- list(); 
    survival <- list()

    for( i in names(studies)){
        response[[i]] <- studies[[i]] %>% filter(Filter_meta_responseMeasured == "Yes") %>% drop_na(Y_best_response_binary)
        survival[[i]] <- studies[[i]]   
    }
    ready = list()
    ready[["survival"]] = survival
    ready[["response"]] = response
    ready
}

In [ ]:
go <- get_ready( studies )

### 2 - Send it!

In [ ]:
saveRDS(go, paste0(TMP_DIR, "exhaustive-ready.Rds"))